# Gaap pipeline 

Use this notebook to run `gaap` photometry on Merian-reduced data.

Make sure that you are in the right environment! When activating the jupyter notebook:

        module load anaconda3/2022.5
        . /scratch/gpfs/am2907/Merian/gaap/lambo/scripts/setup_env_w40.sh
        jupyter notebook

In [15]:
import lsst.daf.butler as dafButler
import numpy as np
import glob
import os, sys
sys.path.append(os.path.join(os.getenv('LAMBO_HOME'), 'lambo/scripts/'))
from hsc_gaap.deploy_gaap_array import deploy_training_job
from hsc_gaap.check_gaap_run import checkRun

---
# Step 1: What patches do we need to reduce?

We want to identify patches that have the necessary merian data products for `gaap` processing and have not already been processed. Patches need to have:

- deepCoadd_ref
- deepCoadd_meas
- deepCoadd_scarletModelData
- deepCoadd_calexp


In [3]:
def findReducedPatches(tract, band="N708", output_collection='DECam/runs/merian/dr1_wide', skymap='hsc_rings_v1'):
        """Returns all merian patches for a given tract that have the necessary data products to run gaap"""

        butler = dafButler.Butler('/projects/MERIAN/repo/')
        dataId = dict(tract=tract, band=band, skymap=skymap)
        
        deepCoadd_ref_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_ref', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        deepCoadd_meas_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_meas', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        deepCoadd_scarletModelData_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_scarletModelData', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        deepCoadd_calexp_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_calexp', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        del butler

        patches = deepCoadd_ref_patches.intersection(deepCoadd_meas_patches, deepCoadd_scarletModelData_patches, deepCoadd_calexp_patches)
        return(np.array(list(patches)))

In [4]:
def findGaapReducedPatches(tract, repo = '/scratch/gpfs/am2907/Merian/gaap/'):
    """Returns all merian patches in a given tract that have already 
       been gaap reduced (have appropriate reduction logs in a given directory)"""

    dir = os.path.join(repo, f"log/{tract}/")
    logs = np.array(glob.glob(dir + "*.o"))
    logs_patches = np.array([log.split("/")[-1].split(".")[0] for log in logs]).astype(int)
    logs, logs_patches = logs[logs_patches.argsort()], logs_patches[logs_patches.argsort()]

    return (logs_patches)

Get a list of all Merian tracts with reduced data, and we will search through them to see which patches fit our criteria:

In [5]:
output_collection = "DECam/runs/merian/dr1_wide"
data_type = "deepCoadd_calexp"
skymap = "hsc_rings_v1"
butler = dafButler.Butler('/projects/MERIAN/repo/', collections=output_collection, skymap=skymap)

In [6]:

patches = np.array([[data_id['tract'], data_id["patch"]] for data_id in butler.registry.queryDataIds (['tract','patch'], datasets=data_type, 
                                                 collections=output_collection, skymap=skymap)])
patches = patches[patches[:, 0].argsort()]

In [7]:
tracts, idx = np.unique(patches[:,0], return_index=True) 
patches_by_tract = np.split(patches[:,1] ,idx[1:])

In [8]:
tracts_n708 = []
for tract in tracts:
    patches = ",".join(findReducedPatches(tract).astype(str))
    if len(patches) > 0:
        tracts_n708.append(tract)

tracts_n708 = np.array(tracts_n708)
print(f"{len(tracts_n708)} tracts with necessary data products")

Now find patches that haven't yet been `gaap` processed:

In [11]:
tracts_n708_nogaap = []
for tract in tracts:
    patches_mer  = findReducedPatches(tract)
    patches_gaap = findGaapReducedPatches(tract)
    if len(set(patches_mer) - set(patches_gaap)) > 0:
        tracts_n708_nogaap.append(tract)
        
print(f"{len(tracts_n708_nogaap)} tracts to be reduced")

If you want, you can see how many patches need to be reduced for each tract:

In [ ]:
# for tract in tracts_n708_nogaap:
#     patches  = list(set(findReducedPatches(tract))- set(findGaapReducedPatches(tract)))
#     if len(patches) > 0:
#         print (f'TRACT:{tract}, {len(patches)}')

---
# Step 2: Download the data

We need to download the HSC data for all of the tracts we need to reduce. *Be warned, this takes a while and uses a lot of storage.*

It is recommended to run the following in a bash screen because depending on how much data you need to download, it can take many hours.

The following will download images for tract 9813 to `/scratch/gpfs/am2907/Merian/gaap/S20A/deepCoadd_calexp/9813` and the blendedness catalogs to `/scratch/gpfs/am2907/Merian/gaap/S20A/gaapTablecxd`:
- Unless `--only_merian=False`, this will only download the patches that have been reduced by Merian.
- You can download all of the Merian-reduced data in one go if you set `--alltracts=True`. Be careful with this, because it is ****lots**** of data!

    screen -L -S downloadtract    
    
    cd /scratch/gpfs/am2907/Merian/gaap
    . lambo/scripts/setup_env_w40.sh
    python3 lambo/scripts/hsc_gaap/download_S20A.py --tract=9813 --outdir="/scratch/gpfs/am2907/Merian/gaap/"


To exit screen do `ctrl a d` and to reattach do `screen -r downloaddata`

---
# Step 3: Make slurm scripts and submit

Write one slurm script for each tract – each of which is a job array with one job for each patch. 
You can submit the scripts as you write them if you want, but beware that there is an upper limit for the number of jobs you can submit at once to the queue.

In [ ]:
# for tract in tracts_n708_nogaap:
#     deploy_training_job(tract, filter_jobs=5,
#                         python_file='lambo/scripts/hsc_gaap/run_gaap.py',
#                         name='gaap', email="am2907@princeton.edu", outname = None, 
#                         repo='/scratch/gpfs/am2907/Merian/gaap', scriptdir="'/scratch/gpfs/am2907/Merian/gaap/", 
#                         submit=True, fixpatches=False)

The gaap reduction will save one catalog for each patch to (for example):

        /scratch/gpfs/am2907/Merian/gaap/S20A/gaapTable/9813/0,0/objectTable_9813_0,0_S20A.fits

---
# Step 4: Check on it!

You can check on the logs while the jobs are running to check for any glaring problems:
- `logs/gaapPhot_array_9813_0.o` 
- `logs/gaap_9813_0.log`

One the jobs are done running (for a given tract), you can check how things went. 

In [ ]:
for tract in tracts_n708_nogaap:
    problems = checkRun(tract)

You might get issues like "Failed for 3 bands" - this could be because HSC images don't exist for all bands. So it might not be an issue you can fix!

---
# Step 4: Merge catalogs

If everything is looking good, you can merge the patch catalogs into a tract-level catalog. 

It's recommended to run this step in a screen in terminal, because it takes some time!

        python3 lambo/scripts/hsc_gaap/compile_catalogs.py --tracts=="[9327,9328,9329,9813,9812]"

This will save a catalog to (for example):
        
        /scratch/gpfs/am2907/Merian/gaap/S20A/gaapTable/9813/objectTable_9813_S20A.fits

If you want to change the columns that are used for the compiled catalog, edit these files:

        lambo/scripts/hsc_gaap/keep_table_columns_gaap.txt
        lambo/scripts/hsc_gaap/keep_table_columns_merian.txt

And you're all done!